# <p style="background-color:green;font-family:newtimeroman;font-size:200%;color:white;text-align:center;border-radius:20px 20px;"><b>Unsupervised Learning Final Project RFM & Cohort Analysis</b></p>
![](https://www.techproeducation.com/logo/headerlogo.svg)

<b>Yeniliklerden ilk siz haberdar olmak istiyorsanız lütfen bizi takip etmeyi unutmayın </b>[YouTube](https://www.youtube.com/c/techproeducation) | [Instagram](https://www.instagram.com/techproeducation) | [Facebook](https://www.facebook.com/techproeducation) | [Telegram](https://t.me/joinchat/HH2qRvA-ulh4OWbb) | [WhatsApp](https://api.whatsapp.com/send/?phone=%2B15853042959&text&type=phone_number&app_absent=0) | [LinkedIn](https://www.linkedin.com/company/techproeducation/mycompany/) |

**1. Proje Tanıtımı**
Bu proje, İngiltere merkezli bir çevrimiçi perakende verisini analiz ederek müşteri davranışlarını anlamayı ve iş stratejileri geliştirmeyi amaçlamaktadır. Proje, aşağıdaki adımları içerir:
- Veri temizleme ve keşifsel veri analizi (**Exploratory Data Analysis - EDA**).
- **RFM Analysis** ile müşteri segmentasyonu.
- **K-Means Clustering** kullanarak müşteri segmentasyonu.
- **Cohort Analysis** ile müşteri tutma oranlarının analizi.

---

**2. Veri Seti Bilgisi**
**Veri Seti Özellikleri:**
- **InvoiceNo**: Fatura numarası (6 haneli, her işlem için benzersiz).
- **StockCode**: Ürün veya stok kodu (5 haneli, her ürün için benzersiz).
- **Description**: Ürün açıklaması.
- **Quantity**: Satılan ürün miktarı.
- **InvoiceDate**: Fatura tarihi ve saati.
- **UnitPrice**: Ürün birim fiyatı (sterlin).
- **CustomerID**: Müşteri numarası (5 haneli).
- **Country**: Müşterinin bulunduğu ülke.

---

**3. Proje Yapısı**
1. **Data Cleaning and Exploratory Data Analysis**
   - Veride eksik değerleri ve gürültüleri tespit etme.
   - Veriyi temizleme ve görselleştirme.
   - Fatura, ürün ve ülke bazında analizler yapma.

2. **RFM Analysis & Customer Segmentation**
   - **Recency** (son satın alma), **Frequency** (satın alma sıklığı) ve **Monetary** (harcama tutarı) değerlerini hesaplama.
   - **RFM Table** oluşturma ve müşterileri segmentlere ayırma.
   - RFM skorlarına dayalı segmentasyon.
   - En iyi müşteriler, kayıp müşteriler gibi kategoriler oluşturma.

4. **K-Means Clustering**
   - Özellik mühendisliği (**Feature Engineering**) ve normalizasyon.
   - Kümeleri belirlemek için **Elbow Method** ve **Silhouette Analysis** yöntemlerini kullanma.
   - Sonuçları görselleştirme.

5. **Cohort Analysis**
   - Satın alma tarihlerini kullanarak müşteri grupları (**cohortlar**) oluşturma.
   - Cohort tablosu ile müşteri tutma oranlarını analiz etme.

---

**5. Çıktılar ve Beklentiler**
- **RFM Analysis**’e dayalı müşteri segmentasyon tablosu.
- **K-Means Clustering** ile görselleştirilmiş müşteri kümeleri.
- **Cohort Analysis** ile müşteri tutma oranlarının görselleştirilmesi.

---

# Exploratory Data Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_excel("Online Retail.xlsx")

In [9]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [19]:
df.shape

(541909, 8)

In [17]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [25]:
df.duplicated().sum()

5268

# RFM Analysis

**RFM Analizi Nedir?**

RFM analizi, müşteri davranışlarını değerlendirmek için kullanılan bir pazarlama teknik ve yöntemidir. 

RFM, Recency (Yenilik), Frequency (Sıklık) ve Monetary (Parasal Değer) kelimelerinin baş harflerinden oluşur.

**RFM bileşenleri:**

- **Recency (Yenilik):** Müşterinin son satın alma işlemini gerçekleştirdiği tarihin ne kadar yakın olduğunu gösterir. Daha yeni satın almış müşteriler, daha yüksek bir recency skoruna sahiptir.
  
- **Frequency (Sıklık):** Müşterinin belirli bir zaman aralığında (genellikle bir yıl içinde) kaç satın alma işlemi gerçekleştirdiğini gösterir. Daha sık alışveriş yapan müşteriler, daha yüksek bir frequency skoruna sahiptir.
  
- **Monetary (Parasal Değer):** Müşterinin belirli bir zaman aralığında harcadığı toplam tutarı ifade eder. Daha yüksek tutarda alışveriş yapan müşteriler, daha yüksek bir monetary skoruna sahiptir.

**RFM Analizi Nasıl Yapılır?**

1. **Müşteri Verisinin Hazırlanması:** İlgili döneme ait müşteri verisi toplanır. Bu veriler genellikle müşteri kimlikleri, satın alma tarihleri ve tutarları içerir.

2. **RFM Skorlarının Hesaplanması:** Her bir müşteri için Recency, Frequency ve Monetary skorları hesaplanır. Bu skorlar genellikle 1 ila 5 arasında bir ölçekte olabilir. Örneğin, 5 en iyi (yani, en yeni, en sık ve en yüksek tutarda) müşteriyi temsil ederken, 1 en kötü müşteriyi temsil eder.

3. **Müşteri Segmentasyonu:** RFM skorlarına göre müşteriler farklı segmentlere ayrılır. Örneğin, RFM skorlarının kombinasyonlarına göre "VIP", "Sadık Müşteri", "Potansiyel Müşteri" gibi segmentler oluşturulabilir.

4. **Pazarlama Stratejilerinin Belirlenmesi:** Her bir müşteri segmenti için uygun pazarlama stratejileri geliştirilir. Örneğin, VIP müşterilere özel indirimler veya sadık müşterilere özel teşvikler sunulabilir.

**RFM Analizinin Faydaları:**

- **Müşteri Segmentasyonu:** RFM analizi, müşterileri farklı gruplara ayırarak pazarlama stratejilerini kişiselleştirmeye olanak tanır.
  
- **Sadık Müşterilerin Tanımlanması:** RFM analizi, sadık müşterileri belirlemek ve onlara özel teşvikler sunmak için kullanılabilir.
  
- **Pazarlama Verimliliğinin Artırılması:** Müşterilerin davranışlarının daha iyi anlaşılması, pazarlama stratejilerinin daha etkili bir şekilde yönetilmesine yardımcı olur.*

# Kmeans

# Cohort Analysis

**Cohort Analizi (Cohort Analysis)**

Cohort analizi, belirli zaman dilimlerinde veya davranışsal özelliklere göre gruplandırılmış kullanıcıların davranışlarını incelemek için kullanılan bir analiz yöntemidir. Özellikle müşteri tutma oranlarını ve kullanıcıların zaman içindeki davranışlarını analiz etmek için faydalıdır.

**Cohort Nedir?**
Cohort, belirli bir ortak özelliği paylaşan bir grup insan veya müşteri demektir. Örneğin:
- **Kaydolma tarihi**: Belirli bir ay içinde kaydolmuş kullanıcılar.
- **İlk alışveriş tarihi**: Belirli bir dönemde ilk alışveriş yapan müşteriler.

**Cohort Analizinin Amaçları:**
1. Müşteri tutma oranlarını analiz etmek.
2. Müşteri davranışlarındaki eğilimleri anlamak.
3. Pazarlama stratejilerinin etkinliğini ölçmek.
4. Zaman içinde müşteri gruplarının nasıl değiştiğini görmek.

**Nasıl Yapılır?**
Cohort Analysis genellikle müşterilerin ilk satın alma veya kayıt tarihine dayalı olarak yapılır. Temel adımlar şunlardır:

1. **Cohort Belirleme**: Analize dahil edilecek müşteri gruplarını tanımlayın. Örneğin, müşterileri kayıt tarihlerine göre aylık olarak gruplara ayırabilirsiniz.

2. **Müşteri Davranışlarının İzlenmesi**: Belirlenen her bir cohort için, müşterilerin zaman içindeki davranışlarını (örneğin, satın alma miktarı, tekrar satın alma oranı) izleyin.

3. **Gelişmelerin Analizi**: Her bir cohort için, zamanla değişen davranışları ve performansı analiz edin. Bu, özellikle cohort'un ilk etkileşimden sonraki davranışlarına ve müşteri yaşam döngüsüne odaklanır.

4. **Segmentasyon ve Karşılaştırma**: Elde edilen verileri analiz edin ve farklı cohort gruplarını karşılaştırın. Bu, belirli pazarlama stratejilerinin veya işletme değişikliklerinin cohort gruplarının davranışlarına nasıl etki ettiğini anlamak için önemlidir.

**Cohort Analizinin Kullanım Alanları:**
- **E-Ticaret:** Müşteri sadakatini ve satın alma eğilimlerini ölçmek.
- **SaaS Şirketleri:** Abonelik iptali (churn) oranını analiz etmek.
- **Mobil Uygulamalar:** Kullanıcıların aktif kalma oranlarını görmek.
  
**Cohort Analizinin Faydaları**

- **Müşteri Davranışlarının Anlaşılması**: Hangi müşteri gruplarının işletme için en değerli olduğunu ve nasıl davrandığını anlamak için değerli bir yol sağlar.

- **Pazarlama Stratejilerinin Optimizasyonu**: Cohort Analysis, pazarlama stratejilerinin etkilerini değerlendirmek ve optimize etmek için kullanılabilir.

- **Sadakat ve Müşteri İlişkileri Yönetimi**: Müşteri sadakati ve ilişkilerinin zaman içinde nasıl değiştiğini anlamak için kullanılabilir.

**Örnek Analiz Soruları:**
- Yeni müşteriler ilk alışverişlerinden sonra ne kadar süre bağlı kalıyor?
- Pazarlama kampanyası belirli bir dönemdeki müşterileri nasıl etkiledi?
- Müşteri tutma oranı zamanla artıyor mu, azalıyor mu?

---

**RFM Analizi ve Cohort Analizi Arasındaki Farklar:**
| **RFM Analizi**                         | **Cohort Analizi**                     |
|------------------------------------------|-----------------------------------------|
| Müşterileri bireysel skorlarla değerlendirir. | Müşterileri gruplar halinde inceler.    |
| Daha çok müşteri segmentasyonu için kullanılır. | Müşteri davranışlarını analiz etmek için uygundur. |
| Hedefi, müşterinin güncel ve geçmiş davranışlarını değerlendirmektir. | Zaman içindeki eğilimleri analiz etmeye odaklanır. |

# <p style="background-color:green;font-family:newtimeroman;font-size:100%;color:white;text-align:center;border-radius:20px 20px;"><b>Faydalı Olması Temennisiyle, Teşekkürler</b></p>
![](https://www.techproeducation.com/logo/headerlogo.svg)

<b>Yeniliklerden ilk siz haberdar olmak istiyorsanız lütfen bizi takip etmeyi unutmayın </b>[YouTube](https://www.youtube.com/c/techproeducation) | [Instagram](https://www.instagram.com/techproeducation) | [Facebook](https://www.facebook.com/techproeducation) | [Telegram](https://t.me/joinchat/HH2qRvA-ulh4OWbb) | [WhatsApp](https://api.whatsapp.com/send/?phone=%2B15853042959&text&type=phone_number&app_absent=0) | [LinkedIn](https://www.linkedin.com/company/techproeducation/mycompany/) |